In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np


In [ ]:
! pip install -U tensorflow-addons

     |████████████████████████████████| 706kB 11.0MB/s 
  Found existing installation: tensorflow-addons 0.8.3
    Uninstalling tensorflow-addons-0.8.3:
      Successfully uninstalled tensorflow-addons-0.8.3


In [ ]:
import tensorflow_addons as tfa

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/dataset/Quora Insincere Question/Quora-Insincere-Question.zip',compression='zip')

In [ ]:
data.columns

Index(['qid', 'question_text', 'target'], dtype='object')

In [ ]:
data.drop(columns=['qid'],inplace=True)

In [ ]:
data['target'].value_counts()

0    1225312
1      80810
Name: target, dtype: int64

In [ ]:
data0=data[data['target']==0]
data1=data[data['target']==1]

In [ ]:
data00=data0[:30000]
data11=data1[:30000]

In [ ]:
new_data=pd.concat([data00,data11])
new_data.reset_index(inplace=True)

In [ ]:
from sklearn.utils import shuffle
new_data = shuffle(new_data)

In [ ]:
new_data.reset_index(inplace=True)
new_data.drop(columns=['level_0','index'],inplace=True)

In [ ]:
import re
def preprocess(w):
  w=w.lower()
  w=re.sub(r"([?.!,%&^%$#@])",r" ",w)
  w=re.sub(r'[" "]+'," ",w)
  w=re.sub(r"[^a-zA-Z?.!,]+"," ",w)
  w=w.strip()
  return w

In [ ]:
ddata=list(map(preprocess,new_data['question_text']))

In [ ]:
len(ddata)

60000

In [ ]:
! pip install transformers
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

     |████████████████████████████████| 1.8MB 10.7MB/s 
     |████████████████████████████████| 890kB 42.4MB/s 
     |████████████████████████████████| 2.9MB 38.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=7b6a38a5d46ccd37926b74753871ee263b9f0763bc689976dd609cf8e27591b1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
train_data=ddata[:50000]
test_data=ddata[50000:]

In [ ]:
inputs=tokenizer(train_data,return_tensors='tf',padding=True,truncation=True,max_length=64)
inputs1=tokenizer(test_data,return_tensors='tf',padding=True,truncation=True,max_length=64)

In [ ]:
inputs11=tokenizer(test_data,return_tensors='tf',padding=True,max_length=64)

In [ ]:
tokenizer.get_vocab

<bound method BertTokenizer.get_vocab of PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})>

In [ ]:
len(test_data)

10000

In [ ]:
inputs11

{'input_ids': <tf.Tensor: shape=(10000, 189), dtype=int32, numpy=
array([[ 101, 2054, 2079, ...,    0,    0,    0],
       [ 101, 2339, 2079, ...,    0,    0,    0],
       [ 101, 2339, 2024, ...,    0,    0,    0],
       ...,
       [ 101, 2339, 2003, ...,    0,    0,    0],
       [ 101, 2129, 2106, ...,    0,    0,    0],
       [ 101, 2054, 2003, ...,    0,    0,    0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(10000, 189), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(10000, 189), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [ ]:
train_data1=inputs['input_ids']
test_data1=inputs1['input_ids']

In [ ]:
next(iter(train_data1))

<tf.Tensor: shape=(64,), dtype=int32, numpy=
array([  101,  2339,  2003, 22035,  2527,  3561,  2007, 10469,  2015,
         102,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0], dtype=int32)>

In [ ]:
tf.range(start=0, limit=10, delta=1)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [ ]:
class Embedding(tf.keras.layers.Layer):
  def __init__(self,projection_dim):
    super(Embedding,self).__init__()
    self.position_embedding=tf.keras.layers.Embedding(input_dim=64,output_dim=projection_dim)
  def call(self,patch):
    positions=tf.range(start=0, limit=patch,delta=1)
    encoded=self.position_embedding(positions)
    return encoded

In [ ]:
def create_att_model():
  inputs=tf.keras.layers.Input(shape=(64,))
  x=tf.keras.layers.Embedding(30522,256,mask_zero=True)(inputs)
 # pos_x=Embedding(256)(inputs)
 # x=x+pos_x
  x11=tf.keras.layers.GRU(256,activation='relu',recurrent_dropout=0.4,return_sequences=True)(x)
  for i in range(8):
    x1=tf.keras.layers.LayerNormalization(epsilon=1e-6)(x11)
    attention_output=tf.keras.layers.MultiHeadAttention(num_heads=8,key_dim=8,dropout=0.1)(x1,x1)
    x2 = tf.keras.layers.Add()([attention_output,x11])
    x3=tf.keras.layers.Dense(256,activation='relu')(x2)
    x4=tf.keras.layers.Add()([x3,x2])
  representation = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x4)
  representation = tf.keras.layers.Flatten()(representation)
  representation = tf.keras.layers.Dropout(0.5)(representation)
  features=tf.keras.layers.Dense(64,activation='relu')(representation)
  logits=tf.keras.layers.Dense(1,activation='sigmoid')(features)
  model=tf.keras.models.Model(inputs=inputs,outputs=logits)
  return model

In [ ]:
model1=create_att_model()

In [ ]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 64, 256)      7813632     input_1[0][0]                    
__________________________________________________________________________________________________
gru (GRU)                       (None, 64, 256)      394752      embedding[0][0]                  
__________________________________________________________________________________________________
layer_normalization_7 (LayerNor (None, 64, 256)      512         gru[0][0]                        
______________________________________________________________________________________________

In [ ]:
optimizer = tfa.optimizers.AdamW(learning_rate=0.1, weight_decay=0.0001)


In [ ]:
model1.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),loss=tf.keras.losses.BinaryCrossentropy(),metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')])

In [ ]:
train_labels=new_data['target'][:50000]

In [ ]:
test_labels=new_data['target'][50000:]

In [ ]:
train_labels

0        1
1        0
2        1
3        0
4        1
        ..
49995    1
49996    1
49997    1
49998    0
49999    1
Name: target, Length: 50000, dtype: int64

In [ ]:
test_data1.shape

TensorShape([10000, 64])

In [ ]:
history=model1.fit(x=train_data1,y=train_labels,batch_size=32,epochs=3,validation_split=0.2)

Epoch 1/3
1250/1250 [==============================] - 210s 166ms/step - loss: 0.6934 - accuracy: 0.5005 - val_loss: 0.6932 - val_accuracy: 0.5053
Epoch 2/3
1250/1250 [==============================] - 207s 166ms/step - loss: 0.6934 - accuracy: 0.4935 - val_loss: 0.6931 - val_accuracy: 0.5053
Epoch 3/3
1250/1250 [==============================] - 207s 166ms/step - loss: 0.6933 - accuracy: 0.4980 - val_loss: 0.6932 - val_accuracy: 0.4947
